# Ansatzes

In [ ]:
import logging
logging.basicConfig(
    format='%(asctime)s-%(levelname)s: %(message)s',
    datefmt='%m/%d/%Y %I:%M:%S %p',
    level=logging.INFO
    #level=logging.DEBUG
)
logger = logging.getLogger('__name__')

In [ ]:
import numpy as np

In [ ]:
import sys
sys.path.append("../../")

In [ ]:
# myQLM qpus
from qat.qpus import PyLinalg, CLinalg
qpu_c = CLinalg()
qpu_p = PyLinalg()
# QLM qpus
from qlmaas.qpus import LinAlg, MPS
qpu_qaass = LinAlg()
qpu_mps = MPS(lnnize =True)

## 1. ansatzes module

One mandatory step for using the Parent Hamiltonian, **PH**, library (see nootebook **02_Using_PH_Class.ipynb**) is computing for a given ansatz its complete state. This is the amplitudes of the state in the computational *n* qubit basis. 

In the *ansatzes* module of the **PH** library some ansatzs were build using **Atos myqlm** software. Additionally several functions that deals with the simulation of a given ansatz using the **Atos myqlm** are provided.

### 1.1 Parent Hamiltonian Github ansatz

The *ansatz_qlm_01* functions implements a **Atos myqlm** version of the ansatz in the github:

https://github.com/FumiKobayashi/Parent_Hamiltonian_as_a_benchmark_problem_for_variational_quantum_eigensolvers

from the original Parent Hamiltonian Papper:

* Kobayashi, F., Mitarai, K., & Fujii, K. (2022). Parent hamiltonian as a benchmark problem for variational quantum eigensolvers. Phys. Rev. A, 105, 052415 (https://doi.org/10.1103%2Fphysreva.105.052415)

We need functions *ansatz_qlm_01* and the function *solve_ansatz* that allows the simulation of a **Atos myqlm** program

In [ ]:
from PH.ansatzes import ansatz_qlm_01, solve_ansatz

We need to provided to the *ansatz_qlm_01* the number of desired qubits and the depth of the circuit. The function returns an *Atos myqlm* objet called *Program* with the implementation of the ansatz and the list of the parameters names of the ansatz

In [ ]:
n_qubits = 12
depth = 3
ansatz_01, theta = ansatz_qlm_01(nqubits=n_qubits, depth=depth)

In [ ]:
# This is a myqlm obcet called Program
ansatz_01

In [ ]:
# Names of the parameters of the ansatz
theta

We can plot the correspondent circuit of the myqlm Program of the ansatz

In [ ]:
circ = ansatz_01.to_circ()
%qatdisplay circ --svg

Now we need to fix the parameter of the circuits in order to obtatain the quantum state of the ansatz

In [ ]:
parameters = list(np.random.rand(len(theta)))
parameters = {v_ : parameters[i] for i, v_ in enumerate(theta)}
print(parameters)

Finally we need to provided the myqlm program, the parameter vector and a *myql* solver to the function *solve_ansatz* for simulating the ansatz and get its final state. IIn this case the provided results is a pandas DataFrame with the complete inforamtion of the ansatz state computed by the simulation.

In [ ]:
pdf = solve_ansatz(ansatz_01, parameters, qpu_mps)

In [ ]:
pdf.head()

### 1.2 Parent Hamiltonian General ansatz

The function *ansatz_qlm_02* implements a generalization of the *ansatz_qlm_01* one. In the *ansatz_qlm_01* all the qubits has the same operations with the same parameters. In the *ansatz_qlm_02* each qubit has the same operations but each operation will have a different  parameter.

In [ ]:
from PH.ansatzes import ansatz_qlm_02

In [ ]:
n_qubits = 12
depth = 3
ansatz_02, theta_02 = ansatz_qlm_02(nqubits=n_qubits, depth=depth)

In [ ]:
circ = ansatz_02.to_circ()
%qatdisplay circ --svg

In [ ]:
parameters_02 = list(np.random.rand(len(theta_02)))

In [ ]:
print("The number of parameters for ansatz 01 is: {}. And for ansatz 02 is :{}".format(
    len(parameters),
    len(parameters_02),    
))

Again we need to fix the parameter of the circuits in order to obtatain the quantum state of the ansatz

In [ ]:
parameters_02 = {v_ : parameters_02[i] for i, v_ in enumerate(theta_02)}
print(parameters_02)

Finally we can use the *solve_ansatz* function for simulating the ansatz.

In [ ]:
pdf_02 = solve_ansatz(ansatz_02, parameters_02, qpu_mps)

In [ ]:
pdf_02.head()

### 1.3 Other ansatzes

For using the **PH** library for computing Parent Hamiltonians the mandatory input will be the list of amplitudes. We can use different ansatzes that are avialable from *Atos myqlm* library.

In [ ]:
from qat.fermion.circuits import make_ldca_circ, make_general_hwe_circ

In [ ]:
nqubit = 8
depth = 3
lda = make_ldca_circ(nqubit, depth)

In [ ]:
%qatdisplay lda --svg

In this case the output of the function is the circuit and not the *Atos myqlm Program*. In this case we can use the function *solving_circuit* from *ansatzes* package for getting the state of the ansatz. 

This functions need:

* Atos myqlm circuit
* number of qubits of the input circuit
* Atos myqlm qpu solver

The output will be a pandas DataFrame with all the information about the sate

In [ ]:
from PH.ansatzes import solving_circuit

In [ ]:
# Fixing the parameter of the ansatz
angles = np.random.rand(len(lda.get_variables()))
lda = lda(
    ** {v: angles[i] for i,v in enumerate(lda.get_variables())})

In [ ]:
pdf_lda = solving_circuit(lda, nqubit, qpu_mps)

In [ ]:
pdf_lda.head()

In [ ]:
circ_02 = make_general_hwe_circ(nqubit, n_cycles=3)

In [ ]:
%qatdisplay circ_02 --svg

In [ ]:
angles_circ_02 = np.random.rand(len(circ_02.get_variables()))
circ_02 = circ_02(
    ** {v: angles_circ_02[i] for i,v in enumerate(circ_02.get_variables())})

In [ ]:
pdf_circ_02 = solving_circuit(circ_02, nqubit, qpu_mps)

In [ ]:
pdf_circ_02